In [2]:
#染色體
#chromosom_value=[[100,200,300],[100,200,300],[0.1,0.2,0.3,0.4,0.5],[0.1,0.2,0.3],[1,2,3],[16,32,64,128,256],["sigmoid","relu","linear"],["sigmoid","relu","linear"]]
#1.batch size：[100,200,300]
#2.epochs ： [100,200,300]
#3.dropout rate：[0.1,0.2,0.3,0.4,0.5]
#4.validation rate ：[0.1,0.2,0.3]
#5.hiden layer ：[1,2,3]
#6.Cell ：[16,32,64,128,256]
#7.livefuncion：["sigmoid","relu","linear"],["sigmoid","relu","linear"]
#染色體長度共8個

#chromosom_value=[[10,20,30],[0.1,0.2,0.3],[16,32,64,128,256],[1,2,3],[100,200,300],[100,200,300],[0.1,0.2,0.3,0.4,0.5],["sigmoid","relu","linear"],["sigmoid","relu","linear"],[0.1,0.2,0.3]]
# 1. TIME_FRAME = [10,20,30] # input資料維度
# 2. TRAIN_RATE = [0.1,0.2,0.3] # 切割train, test資料比例
# 3. CELL_SIZE = ：[16,32,64,128,256] # hidden Layer中memory cells
# 4. LAYERS_NUM = ：[1,2,3] # Layer數
# 5. BATCH_SIZE = ：[100,200,300] # 每次批次投入資料量
# 6. EPOCHS_NUM =  [100,200,300] # 迭代數
# 7. DROPOUT_RATE = [0.1,0.2,0.3,0.4,0.5]3 # Dropout 比例
# 8. activation = '' # 第一層 activation ：["sigmoid","relu","linear"],
# 9. activation = '' # 第二層 activation ：["sigmoid","relu","linear"],
# 10.validation rate ：[0.1,0.2,0.3]

# Fitness Function

In [5]:
def lstm_test(chromosome_one):
    return createrandomList(1,10000)[0]

# GA Function

In [13]:
chromosom_value=[[10,20,30],[0.1,0.2,0.3],[16,32,64,128,256],[1,2,3],[100,200,300],[100,200,300],[0.1,0.2,0.3,0.4,0.5],["sigmoid","relu","linear"],["sigmoid","relu","linear"],[0.1,0.2,0.3]]

import random
def createrandomList(create_number,category):
    list_random = [ random.randint(0,category) for i in range(create_number)]
    return(list_random)

#init create population chromosome(100)--function input：filename & population, output：population_list 
def create_population(population):
    population_list=[]
    for popu in range(population):
        chromosome_create=[]
        for index in chromosom_value:
            chromosome_create.append(index[createrandomList(1,len(index)-1)[0]]) #從chromosom_value隨機產生一個值塞到
        population_list.append(chromosome_create)
    return population_list

def cross(cross_rate,chromosome_1,chromosome_2):
    random_rate=random.random()
    if(random_rate>cross_rate):
        point_1=random.randint(0,len(chromosome_1)-1)
        point_2=random.randint(0,len(chromosome_1)-1)

        while(point_1>=point_2):
            point_1=random.randint(0,len(chromosome_1)-1)
            point_2=random.randint(0,len(chromosome_1)-1)

        print("cross_point1: "+str(point_1))
        print("cross_point2: "+str(point_2))
#         print("cross_len_chromosome_1= "+str(len(chromosome_1)))
#         print("cross_len_chromosome_2= "+str(len(chromosome_2)))

        tmp=0
        for index in range(point_1,point_2+1):
            tmp=chromosome_1[index]
            chromosome_1[index]=chromosome_2[index]
            chromosome_2[index]=tmp
    return chromosome_1

def mutation(mutation_rate,chromosome):
    random_rate=random.random()
    if(random_rate>mutation_rate):
        print("mutation_start")
        point_1=random.randint(0,len(chromosome)-1)#產生二個隨機point1 & point2 為了到時候mutation
        point_2=random.randint(0,len(chromosome)-1)
        while(point_1==point_2):#如果point1 point2 一樣要重新產生
            point_1=random.randint(0,len(chromosome)-1)
            point_2=random.randint(0,len(chromosome)-1)
        print("mu_point1= "+str(point_1))
        print("mu_point2= "+str(point_2))

        tmp=chromosome[point_1]#改變染色體中第point1的值，如果產生的新值與舊的一樣要再重新產生
        #chromosome[point_1]值等於chromosom_value裡面隨機產生的一個值createrandomList(1,len(chromosom_value[point_1])-1)[0]
        chromosome[point_1]=chromosom_value[point_1][createrandomList(1,len(chromosom_value[point_1])-1)[0]]#
        while(tmp==chromosome[point_1]):
            chromosome[point_1]=chromosom_value[point_1][createrandomList(1,len(chromosom_value[point_1])-1)[0]]

        tmp_2=chromosome[point_2]#改變染色體中第point2的值，如果產生的新值與舊的一樣要再重新產生
        chromosome[point_2]=chromosom_value[point_2][createrandomList(1,len(chromosom_value[point_2])-1)[0]]
        while(tmp_2==chromosome[point_2]):
            chromosome[point_2]=chromosom_value[point_2][createrandomList(1,len(chromosom_value[point_2])-1)[0]]
    return chromosome

def calculateY_rank(rank_number,population_list):
    result_all_x_fitness=[]#記錄所有population與產生的y
    best_result=[]#記錄最好的population&y

    for index in population_list:
        fitness_value=lstm_test(index)#calculate fintness_value
        result_all_x_fitness.append([fitness_value,index])
    
    result_rank=[]#record before rank"s x & fitness_value
    for index in range(0,rank_number):
        result_rank.append(sorted(result_all_x_fitness,reverse=True)[index])#sort:big->small
    best_result=result_rank[0]
    
    x_final=[]#記錄前十五名的染色體
    for index in result_rank:
        x_final.append(index[1])
    return [x_final,best_result]

#產生新的X(上一代最好的前15個交配+突變產生新的15個+隨機產生85個新的x)--function
def create_new_x(rank_number,population,cross_rate,mutation_rate,x_final):
    new_x=[]
    #先把前15名的染色體做交配&突變->產生新15個染色體
    for index in range(0,rank_number):
        other_ch=createrandomList(1,rank_number-1)[0]
        while(index==other_ch):
            other_ch=createrandomList(1,rank_number-1)[0] #變矩陣要減1 if index= other_ch -> create other_ch again
        print("i_th chromosome: "+str(index))
        print("other_random_chromosome: "+str(other_ch))
        cross_result=cross(cross_rate,x_final[index],x_final[other_ch])
        new_x.append(mutation(mutation_rate,cross_result))

    #隨機產生第16~100的染色體
    for index in range(0,population-rank_number):
        new_x.append(create_population(1)[0])
    return new_x

# Main：LSTM+GA

In [15]:
#GA + regression ->main
#init parameter

iteration=30
iteration_now=0
population=5
rank_number=3

cross_rate=0.1
mutation_rate=0.1

y_target=20000#fitnessvalue的目標，到達就停止迭代
y_best=-1#目前為止(所有迭代中)最好的fitnessvalue
y_now=-1#當前此迭代中最好的fitnessvalue
x_best=[]

population_list=create_population(population)#產生初始母體染色體

print("Start GA: "+str(Totalstarttime))
while(not(iteration_now>=iteration or y_best>y_target)):
    iteration_now=iteration_now+1
    print("\niteration_now= "+str(iteration_now))
    Y=calculateY_rank(rank_number,population_list) #return X_final(前15名的染色體) & best_result(此代最好的fitness跟染色體)
    y_now=Y[1][0]
    print("y_now= "+str(y_now))
    if y_now>y_best:
        y_best=y_now
        print("y_best= "+str(y_best))
        x_best=Y[1][1]
        #print("x_best= "+str(x_best))
    population_list=create_new_x(rank_number,population,cross_rate,mutation_rate,Y[0])

print("iteration_now= "+str(iteration_now))
print("y_best= "+str(y_best))
print("x_best= "+str(x_best))

print("GA's end time is "+str(datetime.datetime.now()))
print("All time spends:  "+str(datetime.datetime.now()-Totalstarttime))


NameError: name 'Totalstarttime' is not defined